In [15]:
import pandas as pd

# === 1. Load CSV ===
filename = "AGGREGATED_GENERATION_PER_TYPE_GENERATION_15min_2023.csv"
outputfile = "gen_hourly_MW_2023.csv"
df = pd.read_csv(filename, low_memory=False)

# === 2. Parse the MTU column to extract start/end times ===
mtu = df["MTU (CET/CEST)"].str.extract(
    r'(?P<start>\d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2})\s*(?:\((?:CET|CEST)\))?\s*-\s*'
    r'(?P<end>\d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2})\s*(?:\((?:CET|CEST)\))?'
)

df["Start"] = pd.to_datetime(mtu["start"], dayfirst=True, errors="coerce")
df["End"] = pd.to_datetime(mtu["end"], dayfirst=True, errors="coerce")

# === 3. Convert Generation column to numeric ===
gcol = "Generation (MW)"
df[gcol] = pd.to_numeric(df[gcol].astype(str).str.replace(",", "."), errors="coerce")

# === 4. Clean and prepare data ===
df = df.dropna(subset=["Start", gcol]).set_index("Start").sort_index()

# === 5. Define grouping columns (if present) ===
group_cols = [c for c in ["Area", "Production Type"] if c in df.columns]

# === 6. Resample to hourly average power (MW) ===
hourly_avg = (
    df.groupby(group_cols + [pd.Grouper(freq="1h")])[gcol]
      .mean()
      .reset_index()
      .rename(columns={"Start": "datetime", gcol: "Generation (MW)"})
)

# === 7. Reorder columns: put datetime first ===
cols = ["datetime"] + [c for c in hourly_avg.columns if c != "datetime"]
hourly_avg = hourly_avg[cols]

# === 8. Save result (optional) ===
hourly_avg.to_csv(outputfile, index=False)

print(hourly_avg.head())

###
# Load the data
df = pd.read_csv("gen_hourly_MW_all.csv")

dt = pd.to_datetime(df["datetime"], dayfirst=True, errors="coerce")  # catches 01/01/2022
m = dt.isna()
dt.loc[m] = pd.to_datetime(df.loc[m, "datetime"], errors="coerce")   # catches ISO 2022-01-01
if dt.isna().any():
    bad = df.loc[dt.isna(), "datetime"].unique()[:5]
    raise ValueError(f"Unparsed datetimes, examples: {bad}")
df["datetime"] = dt

# Make sure the column names match your CSV exactly
print(df.columns.tolist())  # quick sanity check

# If your column is named 'Producer' (not 'Production'), adjust here:
value_col = "Generation (MW)"
type_col  = "Production Type"

df_wide = (
    df.pivot_table(index="datetime", columns=type_col, values=value_col, aggfunc="sum")
      .sort_index()
      .reset_index()
)

# optional: clean column names
df_wide.columns.name = None
df_wide.columns = [str(c).strip().replace(" ", "_").replace("-", "_") for c in df_wide.columns]

print(df_wide.head())
df_wide.to_csv("gen_hourly_MW_all_wide.csv", index=False)

##
from pathlib import Path

df = pd.read_csv("gen_hourly_MW_all_wide.csv")
df["datetime"] = pd.to_datetime(df["datetime"], errors="raise")

# numeric generation columns (everything except datetime)
gen_cols = [c for c in df.columns if c != "datetime"]

# Sum across generation types to get total per hour
hourly_total = df[gen_cols].sum(axis=1, min_count=1)

out = pd.DataFrame({
    "Hourly_Time_Start": df["datetime"],
    "Country_Area": "Spain (ES)",
    "Average_Hourly_Generation": hourly_total
})

# Optional: format timestamp like your screenshot (DD/MM/YYYY HH:MM)
out["Hourly_Time_Start"] = out["Hourly_Time_Start"].dt.strftime("%Y/%m/%d %H:%M:%S")

out_path = "average_hourly_generation.csv"
out.to_csv("average_hourly_generation.csv", index=False)
print(Path(out_path).resolve())


             datetime    Area Production Type  Generation (MW)
0 2023-01-01 00:00:00  BZN|ES         Biomass            166.0
1 2023-01-01 01:00:00  BZN|ES         Biomass            155.0
2 2023-01-01 02:00:00  BZN|ES         Biomass            148.0
3 2023-01-01 03:00:00  BZN|ES         Biomass            164.0
4 2023-01-01 04:00:00  BZN|ES         Biomass            177.0
['datetime', 'Area', 'Production Type', 'Generation (MW)']
             datetime  Biomass  Fossil_Brown_coal/Lignite  \
0 2022-01-01 00:00:00    526.0                        0.0   
1 2022-01-01 01:00:00    533.0                        0.0   
2 2022-01-01 02:00:00    524.0                        0.0   
3 2022-01-01 03:00:00    532.0                        0.0   
4 2022-01-01 04:00:00    528.0                        0.0   

   Fossil_Coal_derived_gas  Fossil_Gas  Fossil_Hard_coal  Fossil_Oil  \
0                      0.0      3478.0             867.0       135.0   
1                      0.0      3874.0             8